In [ ]:
import requests
import io
import os
import time
from dateutil import parser
import xarray as xr
import regionmask
import geopandas as gp
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cmocean
from datetime import datetime
from glob import glob
import numpy as np
# # SDK
# from cayp import deliver, logging, ExitException
# from cayp.catalog import catalog

In [ ]:
region_bounds = {
    'AUS'  :[-10.3,155,-45,110],
    'TAS1':[-39.5,149.85,-45.15,142.15],
    'VIC1':[-33.95,144.25,-39,140.9],
    'VIC2':[-35.65,150,-39.25,144.25],
    'NSW1':[-28.9,147.65,-36.25,140.9],
    'NSW2':[-28.1,153.75,-32.45,147.65],
    'NSW3':[-32.45,153.3,-37.7,147.65],
    'QLD1':[-23.85,153.75,-29.3,137.9],
    'QLD2':[-8.9,144.4,-23.85,137.9],
    'QLD3':[-13.95,151.5,-23.85,144.4],
    'NT1' :[-10.6,138.15,-19.7,128.9],
    'NT2' :[-19.7,138.15,-26.05,128.9],
    'WA1' :[-13.6,129.1,-23.85,113.1],
    'WA2' :[-23.85,129.1,-35.4,117.85],
    'WA3' :[-23.85,117.85,-35.4,112.85],
    'SA1' :[-25.95,141.05,-38.45,135.5],
    'SA2' :[-25.95,135.5,-35.15,132.65],
    'SAMPLE':[-31.15,134.65,-33.15,132.65]
}

## joining baseline data

In [ ]:
import dask.distributed as dsk

# Start client
clnt = dsk.Client()
clnt

In [ ]:
variable='rsds' # must be either rsds, rhs, tasmax, tasmin or pr

In [ ]:
file_keys={'pr':['AGCD','precip'],'tasmax':['AGCD','tmax'],'tasmin':['AGCD','tmin'],'rhs':['ERA5','hurs'],'rsds':['ERA5','ssrd']}
data=xr.open_mfdataset(f"/g/data/wp00/data/observations/{file_keys[variable][0]}/{file_keys[variable][1]}/daily/*.nc")
if variable=='pr':
    baseline=data.precip.loc["1981-01-01":"2010-12-31"]
elif variable=='tasmax':
    baseline=data.tmax.loc["1981-01-01":"2010-12-31"]
elif variable=='tasmin':
    baseline=data.tmin.loc["1981-01-01":"2010-12-31"]
elif variable=='rhs':
    baseline=data.hurs.loc["1981-01-01":"2010-12-31"]
elif variable=='rsds':
    baseline=data.ssrd.loc["1981-01-01":"2010-12-31"]
baseline=baseline[np.logical_or(baseline['time.month']!=2,baseline['time.day']!=29)].astype('float32')
output=xr.Dataset({variable:baseline})
print(time.strftime("%H:%M:%S", time.localtime())+': saving baseline data')
output.to_netcdf(f'/scratch/wp00/mjk563/{variable}_{file_keys[variable][0]}_historical_1981-2010-daily-mean.nc',format="NETCDF4") # please replace mjk563 with your username

## subsetting/averaging baseline data

In [ ]:
import dask.distributed as dsk

# Start client
clnt = dsk.Client()
clnt

In [ ]:
in_variable='pr'
filein=glob(f'/g/data/wp00/data/QQ-CMIP5/AgDataShop/AUS_daily/{in_variable}_*_historical_1981-2010-daily-mean.nc')[0]
baseline_data=xr.open_dataarray(filein,chunks={'time':'auto'})

In [ ]:
resampled_data=baseline_data.resample(time="M").mean()
output=xr.Dataset({in_variable:resampled_data})
print(time.strftime("%H:%M:%S", time.localtime())+': saving baseline data')
fileout=filein.replace('AUS_daily/','').replace('daily','monthly')
output.to_netcdf(fileout,format="NETCDF4")

In [ ]:
resampled_data=baseline_data.resample(time="Q-NOV").mean()
output=xr.Dataset({in_variable:resampled_data})
print(time.strftime("%H:%M:%S", time.localtime())+': saving baseline data')
fileout=filein.replace('AUS_daily/','').replace('daily','seasonal')
output
output.to_netcdf(fileout,format="NETCDF4")

In [ ]:
resampled_data=baseline_data.resample(time="Y").mean()
output=xr.Dataset({in_variable:resampled_data})
print(time.strftime("%H:%M:%S", time.localtime())+': saving baseline data')
fileout=filein.replace('AUS_daily/','').replace('daily','annual')
output
output.to_netcdf(fileout,format="NETCDF4")

In [ ]:
for in_variable in ['pr','tasmin','tasmax','rhs','rsds']:
    filein=glob(f'/g/data/wp00/data/QQ-CMIP5/AgDataShop/AUS_daily/{in_variable}_*_historical_1981-2010-daily-mean.nc')[0]
    baseline_data=xr.open_dataarray(filein,chunks={'time':'auto'})
    for region in ['TAS1','VIC1','VIC2','NSW1','NSW2','NSW3','QLD1','QLD2','QLD3','NT1','NT2','WA1','WA2','WA3','SA1','SA2','SAMPLE']:
        print(f'{in_variable}, {region}')
        degrees_north =region_bounds[region][0]+.001
        degrees_east  =region_bounds[region][1]+.001
        degrees_south =region_bounds[region][2]-.001
        degrees_west  =region_bounds[region][3]-.001
        subset_data=baseline_data.loc[:,degrees_south:degrees_north,degrees_west:degrees_east]
        fileout=filein.replace('AUS_daily/',f'{region}_daily/').replace('.nc',f'_g-{region}.nc')
        subset_data.to_netcdf(fileout,format="NETCDF4",encoding={subset_data.name:{"zlib": True, "complevel": 5}})